In [ ]:
import numpy as np
import time
import math
import networkx as nx
import matplotlib.pyplot as plt
import argparse
import seaborn as sn
import pandas as pd
from sklearn import metrics
import collections
import torch
import torch.nn as nn
#mlp
import torch.nn.functional as F
import argparse
#mpnn
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader
from dgl import DGLGraph
import sys; sys.argv=['']; del sys
import os
#lib
from utils import *
from load_data import *
parser = argparse.ArgumentParser()
args = parser.parse_args().__dict__
training_setting= {
    'random_seed': 0,
    'batch_size': 32,#64,#
    'node_in_feats': 1,
    'edge_in_feats': 1,
}
args.update(training_setting)

In [ ]:
#MLP - MTL
from network import *
args['GPU_NUM'] = 1
args['data_path_label']='data/label/'
args['device']  = torch.device("cuda:"+str(args['GPU_NUM']) if torch.cuda.is_available() else 'cpu')
# initialize the NN
model = Net()
model.to(args['device'])
args['data_path_x']='data/data_mlp/'
args['model_save_path']='model_saved/mlp/'

# load dataset    
train_loader,val_loader,test_loader=load_dataset(args,network_name='MLP')
#load model
state=torch.load(args['model_save_path']+'model.pth') 
model.load_state_dict(state['model_state_dict'])
cn_p,cn_t,area_p,area_t=predict_mlp(args,model,test_loader,task_name='mtl')
getting_results(cn_p,cn_t,area_p,area_t,show_plot=True,f_name='mlp_mtl')

In [ ]:
#MLP - single, classification
from network import *
args['GPU_NUM'] = 1
args['data_path_label']='data/label/'
args['device']  = torch.device("cuda:"+str(args['GPU_NUM']) if torch.cuda.is_available() else 'cpu')
args['task_name'] = 'classification'
# initialize the NN
model = MLPClassification()
model.to(args['device'])
args['data_path_x']='data/data_mlp/'
args['model_save_path']='model_saved/mlp/'+args['task_name']+'_'
train_loader,val_loader,test_loader=load_dataset(args,network_name='MLP')
#load model
state=torch.load(args['model_save_path']+'model.pth') 
model.load_state_dict(state['model_state_dict'])
#optimizer.load_state_dict(state['optimizer'])

cn_p,cn_t,_,area_t=predict_mlp(args,model,test_loader,task_name=args['task_name'])
acc=np.sum(cn_p==cn_t)/len(cn_p)
print('acc: ',acc)

In [ ]:
#MLP - single, regression
from network import *
args['GPU_NUM'] = 1
args['data_path_label']='data/label/'
args['device']  = torch.device("cuda:"+str(args['GPU_NUM']) if torch.cuda.is_available() else 'cpu')
args['task_name'] = 'regression'
# initialize the NN
model = MLPRegression()
model.to(args['device'])
args['data_path_x']='data/data_mlp/'
args['model_save_path']='model_saved/mlp/'+args['task_name']+'_'
train_loader,val_loader,test_loader=load_dataset(args,network_name='MLP')
#load model
state=torch.load(args['model_save_path']+'model.pth') 
model.load_state_dict(state['model_state_dict'])
#optimizer.load_state_dict(state['optimizer'])

_,cn_t,area_p,area_t=predict_mlp(args,model,test_loader,task_name=args['task_name'])
getting_results_regression(area_t,area_p,show_plot=True,f_name='mlp_reg_')

In [ ]:
### MPNN-sensro geometry, Multitask Learning
from network import *
args['data_path_label']='data/label/'
args['GPU_NUM'] = 1#args['--gpu_num']
args['device']  = torch.device("cuda:"+str(args['GPU_NUM']) if torch.cuda.is_available() else 'cpu')
model = MPNNPredictor(node_in_feats=args['node_in_feats'],edge_in_feats=args['edge_in_feats'])
model.to(args['device'])
args['data_path_x']='data/data_mpnn_default_sensor/'
args['model_save_path']='model_saved/mpnn/default_early_stop.pth'
train_loader,val_loader,test_loader=load_dataset(args,network_name='MPNNSmall')
state=torch.load(args['model_save_path']) 
model.load_state_dict(state['model_state_dict'])

cn_p,cn_t,area_p,area_t=predict_mpnn(args,model,test_loader)
getting_results(cn_p,cn_t,area_p,area_t,show_plot=True,f_name='mpnn_mtl')

In [ ]:
### MPNN-sensro geometry, Single task - Calssification
from dgllife.model import MPNNPredictor
args['task_name'] = 'classification'
args['GPU_NUM'] = 1#args['--gpu_num']
args['device']  = torch.device("cuda:"+str(args['GPU_NUM']) if torch.cuda.is_available() else 'cpu')
args['n_task'] = 40 if args['task_name']=='classification' or args['task_name']=='regression_all' else 1
args['data_path_label']='data/label/'
model = MPNNPredictor(node_in_feats=args['node_in_feats'],
      edge_in_feats=args['edge_in_feats'],
         node_out_feats=64,#32,##node hidden dim
         edge_hidden_feats=128,#32,#
         n_tasks=args['n_task'],
         num_step_message_passing=4,#6,
         num_step_set2set=4,#6,
         num_layer_set2set=2)#3):)
model.to(args['device'])

args['data_path_x']='data/data_mpnn_default_sensor/'
args['model_save_path']='model_saved/mpnn/'+args['task_name']+'_default_sensor_'

train_loader,val_loader,test_loader=load_dataset(args,network_name='MPNNSmall')
state=torch.load(args['model_save_path']+'early_stop.pth') 
model.load_state_dict(state['model_state_dict'])
cn_p,cn_t,_,area_t=predict_mpnn(args,model,test_loader,task_name=args['task_name'])
acc=np.sum(cn_p==cn_t)/len(cn_p)
print('acc: ',acc)

In [ ]:
### MPNN-sensro geometry, Single task - Regression
from dgllife.model import MPNNPredictor
args['task_name'] = 'regression'
args['GPU_NUM'] = 1#args['--gpu_num']
args['device']  = torch.device("cuda:"+str(args['GPU_NUM']) if torch.cuda.is_available() else 'cpu')
args['n_task'] = 40 if args['task_name']=='classification' or args['task_name']=='regression_all' else 1
args['data_path_label']='data/label/'
model = MPNNPredictor(node_in_feats=args['node_in_feats'],
              edge_in_feats=args['edge_in_feats'],
                 node_out_feats=64,#32,##node hidden dim
                 edge_hidden_feats=128,#32,#
                 n_tasks=args['n_task'],
                 num_step_message_passing=4,#6,
                 num_step_set2set=4,#6,
                 num_layer_set2set=2)#3):)
model.to(args['device'])
args['data_path_x']='data/data_mpnn_default_sensor/'
args['model_save_path']='model_saved/mpnn/'+args['task_name']+'_default_sensor_'
train_loader,val_loader,test_loader=load_dataset(args,network_name='MPNNSmall')
state=torch.load(args['model_save_path']+'early_stop.pth') 
model.load_state_dict(state['model_state_dict'])
_,cn_t,area_p,area_t=predict_mpnn(args,model,test_loader,task_name=args['task_name'])
getting_results_regression(area_t,area_p,show_plot=True,f_name='mpnn_reg')